In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install wandb
import numpy as np
import pandas as pd
import wandb
import random
import torch
import torch.nn as nn
from torch import optim
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import torch.nn.functional as F
from tqdm import tqdm
wandb.login()
import csv
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wandb.login(key="17adccf08bc2dbea6a3a448facbf79a0939941e8")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Bhavik More\.netrc


True

In [ ]:
# Step 0: Define the language
#Selecting Marathi language

# Step 1: Create file paths for train, test, and validation data
train_file = f"./aksharantar_sampled/aksharantar_sampled/mar/mar_train.csv"
test_file = f"./aksharantar_sampled/aksharantar_sampled/mar/mar_test.csv"
val_file = f"./aksharantar_sampled/aksharantar_sampled/mar/mar_valid.csv"

# Step 2: Read CSV files and split data into English and Marathi columns
english_train = pd.read_csv(train_file, header=None).iloc[:, 0]
marathi_train = pd.read_csv(train_file, header=None).iloc[:, 1]
english_test = pd.read_csv(test_file, header=None).iloc[:, 0]
marathi_test = pd.read_csv(test_file, header=None).iloc[:, 1]
english_val = pd.read_csv(val_file, header=None).iloc[:, 0]
marathi_val = pd.read_csv(val_file, header=None).iloc[:, 1]

# Step 3: Create character lists and find maximum word lengths
# Create a set of unique characters across all words
english_char_set = set(char for word in english_train for char in word)
marathi_char_set = set(char for word in marathi_train for char in word)

# Sort the sets to create character lists
english_chars = sorted(english_char_set)
marathi_chars = sorted(marathi_char_set)

# Find maximum word lengths in train data
english_max_len = max(len(word) for word in english_train)
marathi_max_len = max(len(word) for word in marathi_train)

# Find maximum word lengths from validation and test data
english_max_len = max(english_max_len, max(len(word) for word in english_val), max(len(word) for word in english_test))
marathi_max_len = max(marathi_max_len, max(len(word) for word in marathi_val), max(len(word) for word in marathi_test))

# Step 4: Convert words to vector representations
english_vectors = []
marathi_vectors = []

for word in english_train:
    vector = [english_chars.index(char) + 1 if char in english_chars else 0 for char in word]
    vector = [len(english_chars) + 1] + vector + [0] * (english_max_len - len(word) + 1)
    english_vectors.append(vector)

for word in marathi_train:
    vector = [marathi_chars.index(char) + 1 if char in marathi_chars else 0 for char in word]
    vector = [len(marathi_chars) + 1] + vector + [0] * (marathi_max_len - len(word) + 1)
    marathi_vectors.append(vector)

# Step 5: Create word matrices
english_matrix = torch.tensor(english_vectors)
marathi_matrix = torch.tensor(marathi_vectors)

# Step 6: Create word matrices for validation and test data
english_matrix_val = torch.tensor([[english_chars.index(char) + 1 if char in english_chars else 0 for char in word] + [0] * (english_max_len - len(word) + 1) + [len(english_chars) + 1] for word in english_val])
english_matrix_test = torch.tensor([[english_chars.index(char) + 1 if char in english_chars else 0 for char in word] + [0] * (english_max_len - len(word) + 1) + [len(english_chars) + 1] for word in english_test])

marathi_matrix_val = torch.tensor([[marathi_chars.index(char) + 1 if char in marathi_chars else 0 for char in word] + [0] * (marathi_max_len - len(word) + 1) + [len(marathi_chars) + 1] for word in marathi_val])
marathi_matrix_test = torch.tensor([[marathi_chars.index(char) + 1 if char in marathi_chars else 0 for char in word] + [0] * (marathi_max_len - len(word) + 1) + [len(marathi_chars) + 1] for word in marathi_test])

In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    """
    Encoder module for sequence-to-sequence models.

    Args:
        input_size (int): Size of the input vocabulary.
        embedding_dim (int): Dimension of the embedding layer.
        hidden_size (int): Size of the hidden state in the recurrent layer.
        num_layers (int): Number of layers in the recurrent layer.
        batch_size (int): Batch size for the input data.
        dropout_prob (float): Dropout probability for regularization.
        bidirectional (bool): Whether to use a bidirectional recurrent layer.
        cell_type (str): Type of recurrent cell ('RNN', 'LSTM', or 'GRU').

    Attributes:
        embedding (nn.Embedding): Embedding layer for input tokens.
        rnn (nn.RNNBase): Recurrent layer (RNN, LSTM, or GRU).
        dropout (nn.Dropout): Dropout layer for regularization.
    """

    def __init__(self, input_size, embedding_dim, hidden_size, num_layers, batch_size, dropout_prob, bidirectional, cell_type):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.dropout = nn.Dropout(dropout_prob)
        self.bidirectional = bidirectional
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.cell_type = cell_type

        self.rnn = self._get_rnn_layer(embedding_dim, hidden_size, num_layers, dropout_prob, bidirectional, cell_type)

    def _get_rnn_layer(self, embedding_dim, hidden_size, num_layers, dropout_prob, bidirectional, cell_type):
        """
        Helper function to create the appropriate recurrent layer.

        Args:
            embedding_dim (int): Dimension of the embedding layer.
            hidden_size (int): Size of the hidden state in the recurrent layer.
            num_layers (int): Number of layers in the recurrent layer.
            dropout_prob (float): Dropout probability for regularization.
            bidirectional (bool): Whether to use a bidirectional recurrent layer.
            cell_type (str): Type of recurrent cell ('RNN', 'LSTM', or 'GRU').

        Returns:
            nn.RNNBase: Recurrent layer (RNN, LSTM, or GRU).
        """
        rnn_class = nn.RNN if cell_type == "RNN" else (nn.LSTM if cell_type == "LSTM" else nn.GRU)
        return rnn_class(embedding_dim, hidden_size, num_layers, dropout=dropout_prob, bidirectional=bidirectional)

    def forward(self, input_sequence):
        """
        Forward pass of the Encoder module.

        Args:
            input_sequence (torch.Tensor): Input sequence of token indices.

        Returns:
            torch.Tensor: Output sequence of hidden states.
            torch.Tensor: Final hidden state(s) of the recurrent layer.
            torch.Tensor: Final cell state(s) of the recurrent layer (for LSTM only).
        """
        embedded = self.dropout(self.embedding(input_sequence))
        outputs, hidden_states = self.rnn(embedded)

        if self.cell_type == "LSTM":
            return outputs, hidden_states[0], hidden_states[1]
        else:
            return outputs, hidden_states

    def initialize_hidden_state(self, device):
        """
        Initialize the hidden state(s) of the recurrent layer.

        Args:
            device (torch.device): Device to create the hidden state(s) on.

        Returns:
            torch.Tensor: Initial hidden state(s) of the recurrent layer.
            torch.Tensor: Initial cell state(s) of the recurrent layer (for LSTM only).
        """
        batch_size = self.batch_size
        hidden_size = self.hidden_size
        num_layers = self.num_layers
        bidirectional = self.bidirectional

        num_directions = 2 if bidirectional else 1
        hidden_state = torch.zeros(num_layers * num_directions, batch_size, hidden_size, device=device)

        if self.cell_type == "LSTM":
            cell_state = torch.zeros(num_layers * num_directions, batch_size, hidden_size, device=device)
            return hidden_state, cell_state
        else:
            return hidden_state

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_layer_size, output_size, decoder_layers, dropout_prob, cell_type, use_attention=False, is_bidirectional=False):
        super(Decoder, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.output_size = output_size
        self.decoder_layers = decoder_layers
        self.dropout = nn.Dropout(dropout_prob)
        self.cell_type = cell_type
        self.use_attention = use_attention
        self.is_bidirectional = is_bidirectional
        self.max_length = len(english_matrix[0])
        self.attention_weights = 0

        self.embedding = nn.Embedding(input_size, embedding_size)
        rnn_input_size = hidden_layer_size if use_attention else embedding_size
        rnn_class = self.get_rnn_class(cell_type)
        self.rnn = rnn_class(rnn_input_size, hidden_layer_size, decoder_layers, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_layer_size, output_size)

        if use_attention:
            self.attn = nn.Linear(hidden_layer_size + embedding_size, self.max_length)
            self.attn_combine = self.get_attention_combine_layer(hidden_layer_size, embedding_size, is_bidirectional)

    def get_rnn_class(self, cell_type):
        if cell_type == "RNN":
            return nn.RNN
        elif cell_type == "LSTM":
            return nn.LSTM
        else:
            return nn.GRU

    def get_attention_combine_layer(self, hidden_layer_size, embedding_size, is_bidirectional):
        if is_bidirectional:
            return nn.Linear(hidden_layer_size * 2 + embedding_size, hidden_layer_size)
        else:
            return nn.Linear(hidden_layer_size + embedding_size, hidden_layer_size)

    def forward(self, input_seq, encoder_outputs, hidden_state, cell_state=None):
        input_seq = input_seq.unsqueeze(0)
        embedded = self.embedding(input_seq)
        embedded = self.dropout(embedded)

        if self.use_attention:
            attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden_state[0]), 1)), dim=1)
            self.attention_weights = attn_weights  # Store attention weights
            attn_applied = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs.permute(1, 0, 2)).squeeze(1)
            combined_input = torch.cat((embedded[0], attn_applied), 1)
            combined_input = self.attn_combine(combined_input).unsqueeze(0)
            combined_input = F.relu(combined_input)
        else:
            combined_input = embedded

        if self.cell_type == "LSTM":
            outputs, (hidden_state, cell_state) = self.rnn(combined_input, (hidden_state, cell_state))
        else:
            outputs, hidden_state = self.rnn(combined_input, hidden_state)

        predictions, hidden_state, cell_state = self.generate_predictions(outputs, hidden_state, cell_state)

        return predictions, hidden_state, cell_state

    def generate_predictions(self, rnn_outputs, rnn_hidden_state, rnn_cell_state=None):
        output_predictions = self.fc(rnn_outputs)
        output_predictions = output_predictions.squeeze(0)

        return (output_predictions, rnn_hidden_state, rnn_cell_state) if self.cell_type == "LSTM" else (output_predictions, rnn_hidden_state, rnn_cell_state)

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_layer_size, device=device)


In [ ]:
class Seq2SeqModel(nn.Module):
    def __init__(self, output_size, cell_type, bidirectional, enc_lyr, dec_lyr, encoder, decoder,attention):
        super(Seq2SeqModel, self).__init__()
        self.output_size = output_size
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        self.enc_lyr = enc_lyr
        self.dec_lyr = dec_lyr
        self.encoder = encoder
        self.attention=attention
        self.decoder = decoder

    def forward(self, source, target , teacher_force_ratio=0.5):
        target_len = target.shape[0]
        batch_size = source.shape[1]

        outputs = torch.zeros(target_len, batch_size, self.output_size).to(source.device)

        encoder_output, hidden, cell = self.encode_sequence(source)
        hidden, cell = self.prepare_decoder_states(hidden, cell)
        outputs,attentions = self.decode_sequence(target, encoder_output, hidden, cell, teacher_force_ratio)

        return outputs,attentions

    def encode_sequence(self, source):
        if self.cell_type == "LSTM":
            encoder_output, hidden, cell = self.encoder(source)
            return encoder_output, hidden, cell
        else:
            encoder_output, hidden = self.encoder(source)
            return encoder_output, hidden, None

    def prepare_decoder_states(self, hidden, cell):
        if self.bidirectional or self.enc_lyr != self.dec_lyr:
          hidden = hidden[self.enc_lyr - 1] + hidden[self.enc_lyr - 1]
          hidden = hidden.repeat(self.dec_lyr,1,1)
          if(self.cell_type == "LSTM"):
              cell = cell[self.enc_lyr - 1] + cell[self.enc_lyr - 1]
              cell = cell.repeat(self.dec_lyr,1,1)
        return hidden, cell


    def decode_sequence(self, tgt, enc_out, hid, cell, teacher_force_ratio):
      batch_size = tgt.shape[1]
      target_len = tgt.shape[0]
      outputs = torch.zeros(target_len, batch_size, self.output_size).to(enc_out.device)
      attentions = []

      timestep = 1
      current_token = tgt[0]

      while timestep < target_len:
          if self.cell_type == "LSTM":
              output, hid, cell = self.decoder(current_token, enc_out, hid, cell)
          else:
              output, hid, cell = self.decoder(current_token, enc_out, hid)
          outputs[timestep] = output

          if(self.attention==True):
            # attentions.append(self.decoder.attn_weights.detach().cpu().numpy())
            attentions.append(self.decoder.attention_weights.detach().cpu().numpy())

          if random.random() < teacher_force_ratio:
              current_token = tgt[timestep] if timestep < target_len - 1 else output.argmax(1)
          else:
              current_token = output.argmax(1)

          timestep += 1

      attentions = np.array(attentions)
      return outputs, attentions


In [ ]:
# Define the sweep configuration
sweep_config = {
    "method": "bayes",
    'metric': {
        'name': 'Val_Accuracy',
        'goal': 'maximize'
    },
    "parameters": {
        "epochs": {"values": [ 10]},
        "lr": {"values": [1e-4]},
        "cell_type": {"values": ["RNN","LSTM", "GRU"]},
        "bidirectional": {"values": [True, False]},
        "enc_lyr": {"values": [1,2]},
        "dec_lyr": {"values": [1,2]},
        "batch_size": {"values": [128]},
        "embedding_dim": {"values": [32,64]},
        "hidden_lyr": {"values": [128,256]},
        "encoder_dropout": {"values": [0, 0.1]},
        "decoder_dropout": {"values": [0, 0.1]},
        "attention": {"values": [True]}
    }
}

In [ ]:
def main():
    # Initialize wandb
    wandb.init()
    config = wandb.config
    wandb.run.name = "_".join([f"{param}:{value}" for param, value in config.items()])
    train_model(**config)

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="DLA3",entity="ch22m009")

# Run the sweep
wandb.agent(sweep_id, function=main,count=50)

Create sweep with ID: a9p0xem2
Sweep URL: https://wandb.ai/ch22m009/DLA3/sweeps/a9p0xem2


wandb: Agent Starting Run: ywf4novg with config:
wandb: 	attention: True
wandb: 	batch_size: 128
wandb: 	bidirectional: True
wandb: 	cell_type: RNN
wandb: 	dec_lyr: 2
wandb: 	decoder_dropout: 0
wandb: 	embedding_dim: 32
wandb: 	enc_lyr: 2
wandb: 	encoder_dropout: 0
wandb: 	epochs: 10
wandb: 	hidden_lyr: 128
wandb: 	lr: 0.0001


Epoch:  1


  0%|                                                                                          | 0/400 [00:00<?, ?it/s]


Run ywf4novg errored:
Traceback (most recent call last):
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\wandb\agents\pyagent.py", line 308, in _run_job
    self._function()
  File "C:\Users\Bhavik More\AppData\Local\Temp\ipykernel_29744\660904229.py", line 6, in main
    train_model(**config)
  File "C:\Users\Bhavik More\AppData\Local\Temp\ipykernel_29744\402864477.py", line 35, in train_model
    output, attentions = model(inp_data.T, target)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

...

Run 3cnu2cpb errored:
Traceback (most recent call last):
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\wandb\agents\pyagent.py", line 308, in _run_job
    self._function()
  File "C:\Users\Bhavik More\AppData\Local\Temp\ipykernel_29744\660904229.py", line 6, in main
    train_model(**config)
  File "C:\Users\Bhavik More\AppData\Local\Temp\ipykernel_29744\402864477.py", line 35, in train_model
    output, attentions = model(inp_data.T, target)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:  1


  0%|                                                                                          | 0/400 [00:00<?, ?it/s]


Run zbqa0c9e errored:
Traceback (most recent call last):
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\wandb\agents\pyagent.py", line 308, in _run_job
    self._function()
  File "C:\Users\Bhavik More\AppData\Local\Temp\ipykernel_29744\660904229.py", line 6, in main
    train_model(**config)
  File "C:\Users\Bhavik More\AppData\Local\Temp\ipykernel_29744\402864477.py", line 35, in train_model
    output, attentions = model(inp_data.T, target)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 